# 02 - Getting and Cleaning Mortality Data
### by Ian Flores Siaca
##### October 2018

## Purpose of this Notebook

#### Learning
* Use of `pandas` `group_by` function.
* Filtering data with string content.

#### Project
* Compute the number of deaths per zone of each municipality.
* Clean the name of the municipalities in the mortality dataset.

## Mortality Data

![](../imgs/usa_today.png)

Hurricane Maria struck Puerto Rico the 20th of September of 2017 at 6:15 a.m. entering through the municipality of Yabucoa as a Category 4 hurricane. According to the governments of Puerto Rico and the U.S. Virgin islands, the cost of the damage is estimated in $102 billion USD1. However, the impact wasn't only economical. Following a lawsuit presented by the Center for Investigative Journalism (CPI, by its spanish initials) and CNN, the Government of Puerto Rico, and more specifically the Demographic Registry, was forced to publish the individual-level data of all the deaths occurred from September 20, 2017 to June 11, 2018. This data is available in the following [Dropbox Link](https://www.dropbox.com/s/k4wrb1ztwu0fwxh/Base%20de%20Datos%20Mortalidad%20en%20PR%20de%20septiembre%2018%20de%202017%20a%2011%20de%20junio%20de%202018%20entregada%20por%20Gobierno%20de%20PR%20al%20CPI.xls?dl=0) from the CPI. 

## Let's code!

#### Load the libraries

In [1]:
import pandas as pd
import os

Here we are downloading the data directly from *Dropbox* and storing it under the `data/` folder in the `mortality.xlsx` file.

In [8]:
os.system("wget --output-document='../data/mortality.xlsx' 'https://www.dropbox.com/s/k4wrb1ztwu0fwxh/Base%20de%20Datos%20Mortalidad%20en%20PR%20de%20septiembre%2018%20de%202017%20a%2011%20de%20junio%20de%202018%20entregada%20por%20Gobierno%20de%20PR%20al%20CPI.xls?dl=0'")

0

In [9]:
data = pd.read_excel("../data/mortality.xlsx")
data.head()

,DeathNumber,Volumen,CertificateNumber,InscriptionYear,RegistryOffice,RegistrationDate,RegistrationDate_Year,RegistrationDate_Month,RegistrationDate_Day,Name,...,FuneralDate,FuneralDate_Year,FuneralDate_Month,FuneralDate_Day,DispositionName,DispositionPlace,FuneralFacilityName,FuneralFacilityLicenseNumber,FuneralFacilityPlace,FuneralFacilityDirectorName
0,20099,34,186,2017,88 - COAMO,2017-09-26,2017,9,26,GLORIA,...,2017-09-22 00:00:00,2017.0,9.0,22.0,PORTA COELI CREMATION PLUS,"BAYAMON, PUERTO RICO",FUNERARIA COAMEÑA,NaN,"COAMO, PUERTO RICO",DANIEL DIAZ AQUINIO
1,20147,34,1286,2017,44 - BAYAMON,2017-10-03,2017,10,3,MARISOL,...,2017-09-30 00:00:00,2017.0,9.0,30.0,MONTE CALVARIO,"CAGUAS, PUERTO RICO",FUNERARIA ALTERNATIVE,NaN,"BAYAMON, PUERTO RICO",HECTOR COLON
2,20160,34,1085,2017,1 - NIVEL CENTRAL,2017-10-03,2017,10,3,RAMON,...,2017-09-26 00:00:00,2017.0,9.0,26.0,MUNICIPAL NUEVO,"ARECIBO, PUERTO RICO",GONZALEZ,NaN,"ARECIBO, PUERTO RICO",ROLANDO VELEZ MENDEZ
3,20168,34,1089,2017,1 - NIVEL CENTRAL,2017-10-03,2017,10,3,GLORIA,...,2017-09-26 00:00:00,2017.0,9.0,26.0,NUEVO JERUSALEN,"UTUADO, PUERTO RICO",UTUADO MEMORIAL,NaN,"UTUADO, PUERTO RICO",OLGA SAVEDA
4,20169,34,1297,2017,44 - BAYAMON,2017-10-03,2017,10,3,LUIS,...,2017-09-30 00:00:00,2017.0,9.0,30.0,NEW HORIZONS,"BAYAMON, PUERTO RICO",FUNERARIA ASENCIO,NaN,"BAYAMON, PUERTO RICO",MELVIN PORTALATIN MONTALVO


#### Filtering the data

* We are using the ResidencePlace column as the column for the Municipality
* There is people that died in Puerto Rico, but there residence place is not Puerto Rico, we want to exclude those data points from the analysis. 
    * `data[data.ResidencePlace.str.contains('PUERTO RICO')]`
* We also want to exclude those people we don't know their municipality of residence.
    * `data[data.ResidenceZone != 'DESCONOCIDO']`

In [4]:
data = data[data.ResidencePlace.str.contains('PUERTO RICO')]
data = data[data.ResidencePlace != 'PUERTO RICO, DESCONOCIDO']
data = data[data.ResidenceZone != 'DESCONOCIDO']

#### Grouping the data
We want to compute the number of deaths in each zone of each municipality:
* Group by `ResidencePlace` and `ResidenceZone`
    * `data.groupby(['ResidencePlace', 'ResidenceZone'])`
* Calculate the number of deaths per zone of each municipality
    * `data.groupby(['ResidencePlace', 'ResidenceZone']).size().reset_index(name='Deaths')`

In [5]:
df_grp = data.groupby(['ResidencePlace', 'ResidenceZone']).size().reset_index(name='Deaths')
df_grp.head()

,ResidencePlace,ResidenceZone,Deaths
0,"PUERTO RICO, ADJUNTAS",RURAL,63
1,"PUERTO RICO, ADJUNTAS",URBANO,50
2,"PUERTO RICO, AGUADA",RURAL,165
3,"PUERTO RICO, AGUADA",URBANO,37
4,"PUERTO RICO, AGUADILLA",RURAL,320


#### Extract the name of the municipalities from the `ResidencePlace` column

In [6]:
municipalities = df_grp.ResidencePlace.tolist()

for i in range(0,len(municipalities)):
    municipalities[i] = municipalities[i].split("PUERTO RICO, ")[1]

In [7]:
df_grp.ResidencePlace = pd.Series(municipalities)
df_grp.head()

,ResidencePlace,ResidenceZone,Deaths
0,ADJUNTAS,RURAL,63
1,ADJUNTAS,URBANO,50
2,AGUADA,RURAL,165
3,AGUADA,URBANO,37
4,AGUADILLA,RURAL,320


#### Save our DataFrame as a CSV for future analysis

In [10]:
df_grp.to_csv("../data/mortality_grouped.csv")

## Optional Questions

1) Did more women died or men?

2) Which month had the higher mortality?

3) Which was the average age of death? Did more infants died or older people?

4) Do you see any patterns in years of education?

5) Do you notice any patter in the ResidencePlace vs the Death Registry municipality? Why is this?

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.